In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display the full content of each column


In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Display the full content of each column
import re
from tqdm import tqdm
tqdm.pandas()  # This adds the tqdm progress bar to pandas operations
from openai import OpenAI
client = OpenAI()
from dotenv import load_dotenv
import json
ret = load_dotenv()
import ast

In [4]:
df1 = pd.read_csv("../data/benz_packge.csv")

In [5]:
df2 = pd.read_csv("../data/benz_rebate.csv")

In [6]:
df1.head(2)

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Default Rebates,Nondefault Rebates,Rate Style,MF 12,RP 12,Live 12,MF 24,RP 24,Live 24,MF 27,RP 27,Live 27,MF 30,RP 30,Live 30,MF 33,RP 33,Live 33,MF 36,RP 36,Live 36,MF 39,RP 39,Live 39,MF 42,RP 42,Live 42,MF 48,RP 48,Live 48,Package Id,Bank Id
0,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,110895.0,104798.0,0.945020,-3000.0,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64763,2
1,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,116005.0,109601.0,0.944795,-3000.0,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64764,2


In [7]:
df1.shape

(1428, 39)

In [8]:
df2.head(2)

,MatrixPackage,Year,Bank,Code,MatrixRateModel,MatrixRateStyle,EndDate,MatirxRebatesSum,AISRebatesSum,Rebate1,RebateDefault1,MatrixValue1,AISValue1,Rebate2,RebateDefault2,MatrixValue2,AISValue2,Rebate3,RebateDefault3,MatrixValue3,AISValue3,Rebate4,RebateDefault4,MatrixValue4,AISValue4,Rebate5,RebateDefault5,MatrixValue5,AISValue5,Rebate6,RebateDefault6,MatrixValue6,AISValue6,Rebate7,RebateDefault7,MatrixValue7,AISValue7,Rebate8,RebateDefault8,MatrixValue8,AISValue8,Rebate9,RebateDefault9,MatrixValue9,AISValue9,Rebate10,RebateDefault10,MatrixValue10,AISValue10,Rebate11,RebateDefault11,MatrixValue11,AISValue11,Rebate12,RebateDefault12,MatrixValue12,AISValue12,Rebate13,RebateDefault13,MatrixValue13,AISValue13
0,2024 Mercedes AMG C43 4MATIC Sedan Sedan,2024,Mercedes Benz Financial,C43W4 C 43W4,C-Class,C43W4 4dr Sdn C43 AMG 4MATIC,3/31/2025 12:00:00 AM,20000.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,MBFS Star Loyalty Bonus Program,N,2000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,9000.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,MBFS Reignite Bonus Program,N,2000.0,0.0,Mercedes Incentive Bonus,Y,7000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024 Mercedes AMG C63 E Performance Sedan,2024,Mercedes Benz Financial,C63W4SE C 63W4SE,C-Class,C63W4SE 4dr Sdn C63 S E PERFORMANCE AMG 4MATIC,3/31/2025 12:00:00 AM,20000.0,0.0,Rising Star Program,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,2000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,9000.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,7000.0,0.0,MBFS Reignite Bonus Program,N,2000.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df2.shape

(125, 61)

In [10]:
df = pd.merge(df1,df2,on='Bank')

In [11]:
df = df[['Package', 'Rate Model', 'Bank', 'MSRP', 'Invoice', 'Formula', 'Buying','MatrixPackage', 'Year', 'Code', 'MatrixRateModel', 'MatrixRateStyle',
       'Default Rebates', 'Nondefault Rebates', 'Rate Style', 'MF 12', 'RP 12',
       'Live 12', 'MF 24', 'RP 24', 'Live 24', 'MF 27', 'RP 27', 'Live 27',
       'MF 30', 'RP 30', 'Live 30', 'MF 33', 'RP 33', 'Live 33', 'MF 36',
       'RP 36', 'Live 36', 'MF 39', 'RP 39', 'Live 39', 'MF 42', 'RP 42',
       'Live 42', 'MF 48', 'RP 48', 'Live 48', 'Package Id', 'Bank Id',
       'EndDate', 'MatirxRebatesSum', 'AISRebatesSum', 'Rebate1',
       'RebateDefault1', 'MatrixValue1', 'AISValue1', 'Rebate2',
       'RebateDefault2', 'MatrixValue2', 'AISValue2', 'Rebate3',
       'RebateDefault3', 'MatrixValue3', 'AISValue3', 'Rebate4',
       'RebateDefault4', 'MatrixValue4', 'AISValue4', 'Rebate5',
       'RebateDefault5', 'MatrixValue5', 'AISValue5', 'Rebate6',
       'RebateDefault6', 'MatrixValue6', 'AISValue6', 'Rebate7',
       'RebateDefault7', 'MatrixValue7', 'AISValue7', 'Rebate8',
       'RebateDefault8', 'MatrixValue8', 'AISValue8', 'Rebate9',
       'RebateDefault9', 'MatrixValue9', 'AISValue9', 'Rebate10',
       'RebateDefault10', 'MatrixValue10', 'AISValue10', 'Rebate11',
       'RebateDefault11', 'MatrixValue11', 'AISValue11', 'Rebate12',
       'RebateDefault12', 'MatrixValue12', 'AISValue12', 'Rebate13',
       'RebateDefault13', 'MatrixValue13', 'AISValue13']]

In [12]:
result_df = df[df['Rate Style']==df['MatrixRateStyle']]

In [13]:
result_df['package_year']=result_df['Package'].apply(lambda x: int(x.split()[0]))

/var/folders/qd/53v0q6g93r5g_dc_bwl8_tlw0000gn/T/ipykernel_40914/767644778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['package_year']=result_df['Package'].apply(lambda x: int(x.split()[0]))


In [14]:
result_df = result_df[result_df.Year==result_df.package_year]

In [15]:
result_df.shape

(1418, 100)

In [19]:
df3 = result_df[['Package','Rate Style']].drop_duplicates()

In [20]:
def car_details_extraction(car,style):
    try:

        response = client.chat.completions.create(
          model="gpt-4o-mini",
          messages=[
            {
              "role": "system",
              "content": [
                {
                  "type": "text",
                  "text": "you are an expert in mercedez cars. extract the model/trim/style/drive_train/body_type as a json from the following information\n\n"
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "New 2025 Mercedes-Benz GLS 450 AWD 4MATIC\n2025\nMercedes-Benz\nGLS\nBase\n\n\n"
                }
              ]
            },
            {
              "role": "assistant",
              "content": [
                {
                  "type": "text",
                  "text": "{\n  \"model\": \"GLS\",\n  \"trim\": \"Base\",\n  \"style\": \"GLS 450\",\n  \"drive_train\": \"AWD 4MATIC\",\n  \"body_type\": \"SUV\"\n}"
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "New 2025 Mercedes-Benz AMG E 53 E 4MATIC + Sedan AWD 4MATIC\n2025\nMercedes-Benz\nE-Class"
                }
              ]
            },
            {
              "role": "assistant",
              "content": [
                {
                  "type": "text",
                  "text": "{\n  \"model\": \"E-Class\",\n  \"trim\": \"AMG E53\",\n  \"style\": \"E 4MATIC+ Sedan\",\n  \"drive_train\": \"AWD 4MATIC\",\n  \"body_type\": \"Sedan\"\n}"
                }
              ]
            },
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "2025 Mercedes C300 Sedan 4Matic Sedan\nC300W4 4dr Sdn C300 4MATIC"
                }
              ]
            },
            {
              "role": "assistant",
              "content": [
                {
                  "type": "text",
                  "text": "{\n  \"model\": \"C-Class\",\n  \"trim\": \"C300\",\n  \"style\": \"C300 4MATIC Sedan\",\n  \"drive_train\": \"AWD 4MATIC\",\n  \"body_type\": \"Sedan\"\n}"
                }
              ]
            },
              {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f'{car} \n {style}'
                }
              ]}

          ],
          response_format={
            "type": "json_object"
          },
          temperature=0,
          max_completion_tokens=2048,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
        print(car,response.choices[0].message.content,"\n","-"*100)
        return response.choices[0].message.content
        
    except:
        print(car)
        return None

In [21]:
df3["car_details"] = df3.apply(lambda x: car_details_extraction(x['Package'],x['Rate Style']),axis=1)

2024 Mercedes AMG EQE SUV {
  "model": "EQE",
  "trim": "AMG",
  "style": "EQE SUV",
  "drive_train": "AWD 4MATIC+",
  "body_type": "SUV"
} 
 ----------------------------------------------------------------------------------------------------
2024 Mercedes AMG EQS Sedan {
  "model": "EQS",
  "trim": "AMG",
  "style": "EQS AMG 4MATIC+ Sedan",
  "drive_train": "AWD 4MATIC+",
  "body_type": "Sedan"
} 
 ----------------------------------------------------------------------------------------------------
2025 Mercedes AMG GT43 2Door Coupe RWD Coupe {
  "model": "AMG GT",
  "trim": "GT43",
  "style": "2Door Coupe",
  "drive_train": "RWD",
  "body_type": "Coupe"
} 
 ----------------------------------------------------------------------------------------------------
2025 Mercedes AMG GT55 Coupe Coupe {
  "model": "AMG GT",
  "trim": "GT55",
  "style": "AMG GT55 Coupe",
  "drive_train": "AWD 4MATIC",
  "body_type": "Coupe"
} 
 ---------------------------------------------------------------------

In [22]:
df3

,Package,Rate Style,car_details
2,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,"{\n ""model"": ""EQE"",\n ""trim"": ""AMG"",\n ""sty..."
1628,2024 Mercedes AMG EQS Sedan,AMGEQSV4 4dr Sdn EQS AMG 4MATIC+,"{\n ""model"": ""EQS"",\n ""trim"": ""AMG"",\n ""sty..."
3577,2025 Mercedes AMG GT43 2Door Coupe RWD Coupe,AMGGT43 2dr Cpe 43,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT43"",\n ..."
4953,2025 Mercedes AMG GT55 Coupe Coupe,AMGGT55 4dr Sdn GT55 4MATIC,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT55"",\n ..."
6829,2025 Mercedes AMG GT63 Coupe 4MATIC Coupe,AMGGT63C4 2dr Cpe 63,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT63"",\n ..."
...,...,...,...
175331,2025 Mercedes AMG SL43 Convertible,SL43R 2dr Conv SL43,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ..."
176207,2025 Mercedes AMG SL43 Convertible,SL43R 2dr Conv SL43 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ..."
176389,2024 Mercedes AMG SL43 Convertible,SL43R4 2dr Conv SL43 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ..."
177515,2024 Mercedes AMG SL55 Convertible,SL55R4 2dr Conv SL55 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL55"",\n ..."


In [ ]:
df

In [78]:
df = df3.copy()

In [79]:

# df['Year'] = df['car_details'].apply(
#             lambda x: json.loads(x)["Year"] if isinstance(x, str) else pd.NA
#         )
df['trim_extracted'] = df['car_details'].apply(
            lambda x: json.loads(x)["trim"] if isinstance(x, str) else pd.NA
        )
df['Model'] = df['car_details'].apply(
            lambda x: json.loads(x)["model"] if isinstance(x, str) else pd.NA
        )
df['drive_train'] = df['car_details'].apply(
            lambda x: json.loads(x)["drive_train"] if isinstance(x, str) else pd.NA
        )
df['body_type'] = df['car_details'].apply(
            lambda x: json.loads(x)["body_type"] if isinstance(x, str) else pd.NA
        )
df['Style'] = df['car_details'].apply(
            lambda x: json.loads(x)["style"] if isinstance(x, str) else pd.NA
        )

In [58]:
df['Make'] = "Mercedes-Benz"

In [59]:
df.head(2)

,Package,Rate Style,car_details,trim_extracted,Model,drive_train,body_type,Style,Make
2,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,"{\n ""model"": ""EQE"",\n ""trim"": ""AMG"",\n ""sty...",AMG,EQE,AWD 4MATIC+,SUV,EQE SUV,Mercedes-Benz
1628,2024 Mercedes AMG EQS Sedan,AMGEQSV4 4dr Sdn EQS AMG 4MATIC+,"{\n ""model"": ""EQS"",\n ""trim"": ""AMG"",\n ""sty...",AMG,EQS,AWD 4MATIC+,Sedan,EQS AMG 4MATIC+ Sedan,Mercedes-Benz


In [60]:
df4 = pd.merge(df[['Make','Package', 'Rate Style', 'trim_extracted', 'Model',
       'drive_train', 'body_type', 'Style']],result_df,on=['Package', 'Rate Style'])

In [61]:
result_df

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,MatrixPackage,Year,Code,MatrixRateModel,MatrixRateStyle,Default Rebates,Nondefault Rebates,Rate Style,MF 12,RP 12,Live 12,MF 24,RP 24,Live 24,MF 27,RP 27,Live 27,MF 30,RP 30,Live 30,MF 33,RP 33,Live 33,MF 36,RP 36,Live 36,MF 39,RP 39,Live 39,MF 42,RP 42,Live 42,MF 48,RP 48,Live 48,Package Id,Bank Id,EndDate,MatirxRebatesSum,AISRebatesSum,Rebate1,RebateDefault1,MatrixValue1,AISValue1,Rebate2,RebateDefault2,MatrixValue2,AISValue2,Rebate3,RebateDefault3,MatrixValue3,AISValue3,Rebate4,RebateDefault4,MatrixValue4,AISValue4,Rebate5,RebateDefault5,MatrixValue5,AISValue5,Rebate6,RebateDefault6,MatrixValue6,AISValue6,Rebate7,RebateDefault7,MatrixValue7,AISValue7,Rebate8,RebateDefault8,MatrixValue8,AISValue8,Rebate9,RebateDefault9,MatrixValue9,AISValue9,Rebate10,RebateDefault10,MatrixValue10,AISValue10,Rebate11,RebateDefault11,MatrixValue11,AISValue11,Rebate12,RebateDefault12,MatrixValue12,AISValue12,Rebate13,RebateDefault13,MatrixValue13,AISValue13,package_year
2,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,110895.0,104798.0,0.945020,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64763,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024
127,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,116005.0,109601.0,0.944795,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64764,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024
252,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,117270.0,110790.0,0.944743,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64765,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024
377,2024 Mercedes AMG EQE SUV,AMG EQE,Mercedes Benz Financial,117780.0,111270.0,0.944727,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQE

In [62]:
df4.rename({"Rate Style":'Trim'},inplace=True,axis=1)

In [63]:
df4

,Make,Package,Trim,trim_extracted,Model,drive_train,body_type,Style,Rate Model,Bank,MSRP,Invoice,Formula,Buying,MatrixPackage,Year,Code,MatrixRateModel,MatrixRateStyle,Default Rebates,Nondefault Rebates,MF 12,RP 12,Live 12,MF 24,RP 24,Live 24,MF 27,RP 27,Live 27,MF 30,RP 30,Live 30,MF 33,RP 33,Live 33,MF 36,RP 36,Live 36,MF 39,RP 39,Live 39,MF 42,RP 42,Live 42,MF 48,RP 48,Live 48,Package Id,Bank Id,EndDate,MatirxRebatesSum,AISRebatesSum,Rebate1,RebateDefault1,MatrixValue1,AISValue1,Rebate2,RebateDefault2,MatrixValue2,AISValue2,Rebate3,RebateDefault3,MatrixValue3,AISValue3,Rebate4,RebateDefault4,MatrixValue4,AISValue4,Rebate5,RebateDefault5,MatrixValue5,AISValue5,Rebate6,RebateDefault6,MatrixValue6,AISValue6,Rebate7,RebateDefault7,MatrixValue7,AISValue7,Rebate8,RebateDefault8,MatrixValue8,AISValue8,Rebate9,RebateDefault9,MatrixValue9,AISValue9,Rebate10,RebateDefault10,MatrixValue10,AISValue10,Rebate11,RebateDefault11,MatrixValue11,AISValue11,Rebate12,RebateDefault12,MatrixValue12,AISValue12,Rebate13,RebateDefault13,MatrixValue13,AISValue13,package_year
0,Mercedes-Benz,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,AMG,EQE,AWD 4MATIC+,SUV,EQE SUV,AMG EQE,Mercedes Benz Financial,110895.0,104798.0,0.945020,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64763,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024
1,Mercedes-Benz,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,AMG,EQE,AWD 4MATIC+,SUV,EQE SUV,AMG EQE,Mercedes Benz Financial,116005.0,109601.0,0.944795,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64764,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1+),N,13000.0,0.0,NaN,NaN,NaN,NaN,2024
2,Mercedes-Benz,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,AMG,EQE,AWD 4MATIC+,SUV,EQE SUV,AMG EQE,Mercedes Benz Financial,117270.0,110790.0,0.944743,-3000.0,2024 Mercedes AMG EQE SUV,2024,AMGEQEX4 AMG EQEX4,EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,NaN,NaN,NaN,NaN,NaN,0.00042,51.0,True,0.00042,50.0,True,0.00042,49.0,True,0.00042,48.0,True,0.00042,47.0,True,0.00042,45.0,True,0.00042,44.0,True,0.00042,38.0,True,64765,2,3/31/2025 12:00:00 AM,37500.0,0.0,Mercedes-Benz Lease Cash,Y,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Incentive Bonus,Y,11000.0,0.0,Mercedes Lease Bonus Cash,Y,7500.0,0.0,Mercedes Three Payment Loyalty Accelerator Pro...,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Rising Star Program,N,0.0,0.0,Mercedes Five Payment Loyalty Accelerator Prog...,N,0.0,0.0,MBFS Star Loyalty Bonus Program,N,3000.0,0.0,MBFS Reignite Bonus Program,N,3000.0,0.0,Mercedes Dealership Employee One Plus (DE1

In [64]:
df4.to_csv("../data/benz_merged.csv",index=False)

In [65]:
df = df4.copy()

In [66]:
df['Selling Price'] = df['Invoice']-df['Buying']

In [67]:
df['Adjusted Cap Cost'] = df['Selling Price']

In [68]:
acura_df = pd.read_csv("../data/merged_data_v4.csv")

In [69]:
final_df = pd.concat([acura_df,df])

In [70]:
final_df[acura_df.columns].to_csv("../data/merged_data_v5.csv")

In [72]:
new_df = final_df[acura_df.columns]

In [75]:
new_df[new_df.Make=="Mercedes-Benz"].sample()

,Package,Rate Model,Bank,MSRP,Invoice,Formula,Buying,Selling Price_36,Selling Price,Default Rebates,Nondefault Rebates,Default Rebates_36,Total_Rebates,Adjusted Cap Cost,Trim,MF 24,RP 24,MF 27,RP 27,MF 30,RP 30,MF 33,RP 33,MF 36,RP 36,MF 39,RP 39,MF 42,RP 42,MF 48,RP 48,Package Id,Bank Id,car_details,Year,Make,Model,Style,body_type,drive_train,code
469,2024 Mercedes EQE350 4Matic Sedan Sedan,EQE350 4Matic Sedan,Mercedes Benz Financial,93880.0,88008.0,0.937452,-2000.0,NaN,90008.0,NaN,NaN,NaN,NaN,90008.0,EQE350V4 4dr Sdn EQE350 4MATIC+,0.0011,51.0,0.0011,51.0,0.0011,51.0,0.0011,50.0,0.0011,49.0,0.0011,46.0,0.0011,44.0,0.0011,40.0,64718,2,NaN,2024,Mercedes-Benz,EQE,EQE350 4MATIC+ Sedan,Sedan,AWD 4MATIC,NaN


In [80]:
df

,Package,Rate Style,car_details,trim_extracted,Model,drive_train,body_type,Style
2,2024 Mercedes AMG EQE SUV,AMGEQEX4 4dr Wgn EQE AMG 4MATIC+,"{\n ""model"": ""EQE"",\n ""trim"": ""AMG"",\n ""sty...",AMG,EQE,AWD 4MATIC+,SUV,EQE SUV
1628,2024 Mercedes AMG EQS Sedan,AMGEQSV4 4dr Sdn EQS AMG 4MATIC+,"{\n ""model"": ""EQS"",\n ""trim"": ""AMG"",\n ""sty...",AMG,EQS,AWD 4MATIC+,Sedan,EQS AMG 4MATIC+ Sedan
3577,2025 Mercedes AMG GT43 2Door Coupe RWD Coupe,AMGGT43 2dr Cpe 43,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT43"",\n ...",GT43,AMG GT,RWD,Coupe,2Door Coupe
4953,2025 Mercedes AMG GT55 Coupe Coupe,AMGGT55 4dr Sdn GT55 4MATIC,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT55"",\n ...",GT55,AMG GT,AWD 4MATIC,Coupe,AMG GT55 Coupe
6829,2025 Mercedes AMG GT63 Coupe 4MATIC Coupe,AMGGT63C4 2dr Cpe 63,"{\n ""model"": ""AMG GT"",\n ""trim"": ""GT63"",\n ...",GT63,AMG GT,AWD 4MATIC,Coupe,AMG GT63 Coupe 4MATIC
...,...,...,...,...,...,...,...,...
175331,2025 Mercedes AMG SL43 Convertible,SL43R 2dr Conv SL43,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ...",AMG SL43,SL,RWD,Convertible,SL43 Convertible
176207,2025 Mercedes AMG SL43 Convertible,SL43R 2dr Conv SL43 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ...",AMG SL43,SL,RWD,Convertible,SL43 Convertible
176389,2024 Mercedes AMG SL43 Convertible,SL43R4 2dr Conv SL43 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL43"",\n ...",AMG SL43,SL,RWD,Convertible,SL43 Convertible
177515,2024 Mercedes AMG SL55 Convertible,SL55R4 2dr Conv SL55 AMG,"{\n ""model"": ""SL"",\n ""trim"": ""AMG SL55"",\n ...",AMG SL55,SL,RWD,Convertible,SL55 Convertible
